In [ ]:
!wget -N https://github.com/opencv/opencv/raw/3.4.0/samples/dnn/face_detector/deploy.prototxt
!wget -N https://github.com/opencv/opencv_3rdparty/raw/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel

--2021-09-05 08:50:48--  https://github.com/opencv/opencv/raw/3.4.0/samples/dnn/face_detector/deploy.prototxt
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/opencv/opencv/3.4.0/samples/dnn/face_detector/deploy.prototxt [following]
--2021-09-05 08:50:48--  https://raw.githubusercontent.com/opencv/opencv/3.4.0/samples/dnn/face_detector/deploy.prototxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28092 (27K) [text/plain]
Saving to: ‘deploy.prototxt’

deploy.prototxt     100%[===================>]  27.43K  --.-KB/s    in 0.003s  

Last-modified header missing -- time-stamps turned off.
2021-09-0

In [ ]:
import sys, os
import numpy as np
from math import *
import math
from PIL import Image
from matplotlib import pyplot
from matplotlib.patches import Rectangle, Circle
import matplotlib.pyplot as plt
import cv2
import random
import dlib
import imutils

In [ ]:
def bb_intersection_over_union(box1, box2):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(box1[0], box2[0])
	yA = max(box1[1], box2[1])
	xB = min(box1[2], box2[2])
	yB = min(box1[3], box2[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	box1Area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
	box2Area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(box1Area + box2Area - interArea)
	# return the intersection over union value
	return iou

In [ ]:
def calculate_iou(gtbox,SSDbox):
  no_counter = 0
  text = img_file + "\n" + str(num_faces) + "," + str(len(SSDbox)) + "\n"
  for boxA in gtbox:
    if len(SSDbox)==0:
      text += ','.join([str(elem) for elem in boxA]) + '/' + "None" + '/' + "0.0" + '/' + "0.0" + '\n'
      no_counter += 1
    else:
      test_iou = []
      for boxB in SSDbox:
        #print(boxB[0])
        test_iou.append(bb_intersection_over_union(boxA, boxB[0]))
      #FN
      if max(test_iou) == 0.0:
        text += ','.join([str(elem) for elem in boxA]) + "/" + "None" + '/' + "0.0" + "/" + str(max(test_iou)) + "\n"
        no_counter += 1
      #TP
      else:
        for i,u in enumerate(test_iou):
          if u==max(test_iou):
            index=i
            text += ','.join([str(elem) for elem in boxA]) + "/" + ','.join([str(elem) for elem in SSDbox[i][0]]) + "/" + str(SSDbox[i][1]) + "/" + str(max(test_iou)) + "\n"
            break
    """#FP
    for boxC in SSDbox:
      test_iou=[]
      for boxD in gtbox:
        test_iou.append(bb_intersection_over_union(boxC, boxD))
        if max(test_iou)==0.0:
          text += 'None' + '/' + ','.join([str(elem) for elem in boxC])' + '/' + str(correct_iou) + '\n' """
  
  #print(text)
  return text

In [ ]:
folds = ["01","02","03","04","05","06","07","08","09","10"]
for fold_num in folds:

  rect_filename = '/content/drive/MyDrive/paper_face_recognitioin/FDDB-folds/rect/FDDB-fold-{f_name}-rectList.txt'.format(f_name = fold_num)
  iou_filename = '/content/drive/MyDrive/paper_face_recognitioin/FDDB-folds/iou/SSD/FDDB-fold-{f_name}-SSD-IOU.txt'.format(f_name = fold_num)

  with open(rect_filename) as f:
    lines = [line.rstrip('\n') for line in f]

  counter = 0
  f = open(iou_filename,'w')
  while counter < len(lines):
    img_file = lines[counter] 
    image = cv2.imread(img_file)
    num_faces = int(lines[counter+1])
    gtbox = []
    for j in range(num_faces):
      gt = [int(s) for s in lines[counter+j+2].split(',')]
      gtbox.append(gt)

    SSDrect = []
    Confidence = []
    detector = cv2.dnn.readNetFromCaffe("deploy.prototxt" , "res10_300x300_ssd_iter_140000.caffemodel")
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    detector = cv2.dnn.readNetFromCaffe("deploy.prototxt" , "res10_300x300_ssd_iter_140000.caffemodel")
    detector.setInput(blob)
    detections = detector.forward()
    for i in range(0, detections.shape[2]):
      confidence = detections[0, 0, i, 2]
      if confidence != 0.0:
        Confidence.append(confidence)
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        SSD=[startX, startY, endX, endY]
        SSDrect.append(SSD)
    SSDbox = list(zip(SSDrect,Confidence))

    print(SSDbox)

    text = calculate_iou(gtbox,SSDbox)
    
    f.write(text)
    counter = counter + num_faces + 2

  try:
    print(no_counter)
  except: 
    pass
  print(fold_num,"is done")
  f.close()

[([2, 18, 62, 106], 0.9999682), ([259, 95, 330, 186], 0.9999579), ([167, 86, 231, 180], 0.99974865), ([357, 142, 416, 215], 0.9750699), ([1873, 1382, 2179, 1719], 0.12323278), ([71, 1382, 373, 1721], 0.12122747), ([1871, 11, 2176, 340], 0.120534), ([239, 227, 292, 290], 0.11631681), ([29, 151, 48, 180], 0.11560592), ([326, 183, 362, 216], 0.114619486), ([134, 186, 160, 226], 0.11080786), ([22, 135, 39, 162], 0.109465614), ([10, 41, 438, 301], 0.10707647), ([188, 270, 206, 294], 0.10645495), ([166, 204, 181, 232], 0.10633652), ([52, 151, 66, 171], 0.10632586), ([25, 146, 41, 176], 0.105510056), ([44, 152, 60, 178], 0.10459106), ([199, 2, 255, 97], 0.10445003), ([172, 196, 187, 218], 0.10410043), ([178, 188, 197, 223], 0.10340908), ([176, 186, 191, 209], 0.10283189), ([379, 290, 429, 344], 0.10281355), ([391, 319, 419, 344], 0.10255862), ([217, 185, 270, 246], 0.102535866), ([264, 10, 297, 54], 0.10252735), ([314, 0, 425, 95], 0.10155619), ([187, 285, 206, 310], 0.101527065), ([161, 207,